Use Case 9
==========

Problem Definition:
-------------------

A climate scientist wishes to analyse potential correlations between *Ozone* and *Cloud* ECVs.

Required Toolbox Features:
--------------------------

* Access to and ingestion of ESA CCI Ozone and Cloud data (Atmosphere Mole Content of Ozone and Cloud Cover)
* Geometric adjustments (coregistration)
* Spatial (point, polygon) and temporal subsetting
* Visualisation of time series
* Correlation analysis, scatter-plot of correlation statistics, saving of image and correlation statistics on disk (format options)

Ingest data and create datasets
-------------------------------

In [ ]:
from cate.core.ds import DATA_STORE_REGISTRY
import cate.ops as ops

In [ ]:
import xarray as xr

print(xr.__version__)

In [ ]:
data_store = DATA_STORE_REGISTRY.get_data_store('esa_cci_odp')

In [ ]:
data_store.query('ozone')

In [ ]:
data_store.query('cloud')

In [ ]:
#cc = ops.open_dataset('esacci.CLOUD.mon.L3C.CLD_PRODUCTS.AVHRR.NOAA-17.AVHRR_NOAA.1-0.r1',
#                      '2007-01-01, 2007-12-31')
cc = ops.open_dataset('esacci.CLOUD.mon.L3C.CLD_PRODUCTS.AVHRR.NOAA-16.AVHRR_NOAA.1-0.r1',
                      '2007-01-01, 2007-03-01')

In [ ]:
oz = ops.open_dataset('esacci.OZONE.mon.L3.NP.multi-sensor.multi-platform.MERGED.fv0002.r1',
                      '2007-01-01, 2007-12-21')

Filter datasets to select the desired variables
-----------------------------------------------

In [ ]:
cc_tot = ops.select_var(cc, 'cc_total')
oz_tot = ops.select_var(oz, 'O3_du_tot')

In [ ]:
oz_tot

Plot the first time slice of the dataset

In [ ]:
%matplotlib inline
ops.plot_map(oz_tot, var='O3_du_tot', time=0, file='/home/ccitbx/Desktop/fig1.png')

In [ ]:
ops.plot_map(cc_tot, var='cc_total', time=0, file='/home/ccitbx/Desktop/fig2.png')

Co-register datasets by resampling
----------------------------------

In [ ]:
print(cc_tot['cc_total'].shape)
print(oz_tot['O3_du_tot'].shape)

In [ ]:
cc_tot_res = ops.coregister(oz_tot, cc_tot)

In [ ]:
print(cc_tot_res['cc_total'].shape)
print(oz_tot['O3_du_tot'].shape)

In [ ]:
ops.plot_map(cc_tot_res, var='cc_total', time=0, file='/home/ccitbx/Desktop/fig3.png')

Select the desired spatial region
---------------------------------

In [ ]:
africa = '-20.0, -40.0, 60.0, 40.0'
# 'lon_min, lat_min, lon_max, lat_max'
cc_tot_africa = ops.subset_spatial(cc_tot_res, africa)
oz_tot_africa = ops.subset_spatial(oz_tot, africa)

In [ ]:
ops.plot_map(cc_tot_africa, var='cc_total', time=0, file='/home/ccitbx/Desktop/fig4.png')

In [ ]:
ops.plot_map(cc_tot_africa, var='cc_total', time=0, 
                 region=africa, file='/home/ccitbx/Desktop/fig5.png')

Select the desired temporal region
----------------------------------

In [ ]:
cc_tot_africa.time.values

In [ ]:
from jdcal import jd2gcal, MJD_0
from datetime import datetime
tuples = [jd2gcal(x-MJD_0, MJD_0) for x in cc_tot_africa.time.values]
cc_tot_africa.time.values = [datetime(x[0], x[1], x[2]) for x in tuples]
cc_tot_africa.time.attrs['long_name'] = 'time'
cc_tot_africa.time.attrs['calendar'] = 'standard'
cc_tot_africa.time.attrs['test'] = 'test'
cc_tot_africa.time

In [ ]:
cc_tot_africa.time.values = [ 2454101.5,  2454132.5,  2454160.5,  2454191.5,  2454221.5,
        2454252.5,  2454282.5,  2454313.5,  2454344.5,  2454374.5,
        2454405.5,  2454435.5]

In [ ]:
cc_tot_africa.time

In [ ]:
cc_tot_janoct = ops.subset_temporal(cc_tot_africa, '2007-01-01, 2007-10-30')
oz_tot_janoct = ops.subset_temporal(oz_tot_africa, '2007-01-01, 2007-10-30')

In [ ]:
print(cc_tot_janoct.time)

In [ ]:
print(oz_tot_janoct.time)

Retrieve and plot timeseries
----------------------------

In [ ]:
oz_ts_point = ops.tseries_point(oz_tot_janoct, lat=50., lon=50.)
cc_ts_point = ops.tseries_point(cc_tot_janoct, lat=50., lon=50.)
oz_ts_mean = ops.tseries_mean(oz_tot_janoct, var='O3_du_tot')
cc_ts_mean = ops.tseries_mean(cc_tot_janoct, var='cc_total')

In [ ]:
print(oz_ts_mean)

In [ ]:
print(cc_ts_mean)

In [ ]:
ops.plot_1D(cc_ts_mean, 'cc_total', file='/home/ccitbx/Desktop/fig6.png')

In [ ]:
ops.plot_1D(oz_ts_mean, 'O3_du_tot', file='/home/ccitbx/Desktop/fig7.png')

Do correlation analysis
-----------------------

In [ ]:
correlation = ops.pearson_correlation(cc_ts_mean, oz_ts_mean, 'cc_total', 'O3_du_tot',
                                          file="/home/ccitbx/Desktop/correlation.txt")
correlation

In [ ]:
correlation = ops.pearson_correlation(cc_tot_janoct, oz_tot_janoct, 'cc_total', 'O3_du_tot')

In [ ]:
correlation

In [ ]:
ops.plot_map(correlation, var='corr_coef', 
                 lat_min=-40., lat_max=40., lon_min=-20., lon_max=60., file='/home/ccitbx/Desktop/fig8.png')